In [1]:
import time, requests, urllib3, os, traceback
import pandas as pd
from guid import GUID
from flask import Flask, request, session, render_template, redirect, url_for, jsonify, send_from_directory
from flask_cors import CORS
import urllib
from itertools import product, combinations
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from pyvis.network import Network
import sqlalchemy as sa
import graphistry
import graph_visualization as gv

params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()


In [2]:
x = "LANSOPRAZOLE|ATORVASTATINE|POTASSIUM|FUROSEMIDE|ACETYLSALICYLIC"
x = x.split("|")
x

# %LANSOPRAZOLE% OR %ATORVASTATINE% OR %POTASSIUM% OR %FUROSEMIDE% OR %ACETYLSALICYLIC%'

['LANSOPRAZOLE', 'ATORVASTATINE', 'POTASSIUM', 'FUROSEMIDE', 'ACETYLSALICYLIC']

In [3]:
sql_x = []
for items in x:
    x_ = '%' + items + '%'
    sql_x.append(x_)

sql_x = ' OR '.join(sql_x)

In [4]:
#get ids, atcs for every drug in the list

qry = '''
    SELECT Drug.name AS [Drug Name], Drug.synonyms, Drug.atc AS [atc], SideEffect.name AS [SideEffect]
    FROM drug_nodes Drug, drug_sideeffect_edges DSE, sideeffect_nodes SideEffect
    WHERE MATCH(Drug-(DSE)->SideEffect)
    AND CONTAINS(synonyms, '{0}')
    '''.format(sql_x)
    
df_x = pd.read_sql_query(qry, cnxn)

df_x

,Drug Name,synonyms,atc,SideEffect
0,Acetylsalicylic acid,2-Acetoxybenzenecarboxylic acid | 2-Acetoxyben...,B01AC06,Respiratory alkaloses
1,Acetylsalicylic acid,2-Acetoxybenzenecarboxylic acid | 2-Acetoxyben...,B01AC06,Systemic anaphylactic reaction
2,Acetylsalicylic acid,2-Acetoxybenzenecarboxylic acid | 2-Acetoxyben...,B01AC06,Unspecified anaemia
3,Acetylsalicylic acid,2-Acetoxybenzenecarboxylic acid | 2-Acetoxyben...,B01AC06,Angio-oedema
4,Acetylsalicylic acid,2-Acetoxybenzenecarboxylic acid | 2-Acetoxyben...,B01AC06,Bronchitic asthma
...,...,...,...,...
664,Lansoprazole,Lansoprazol | Lansoprazole | Lansoprazolum,A02BC03,Feeling abnormal
665,Lansoprazole,Lansoprazol | Lansoprazole | Lansoprazolum,A02BC03,Dysphonia
666,Lansoprazole,Lansoprazol | Lansoprazole | Lansoprazolum,A02BC03,Procedural pain
667,Lansoprazole,Lansoprazol | Lansoprazole | Lansoprazolum,A02BC03,Somnolence


In [7]:
for idx, row in df_x.iterrows():
    print(row.to_dict())

{'Drug Name': 'Acetylsalicylic acid', 'synonyms': '2-Acetoxybenzenecarboxylic acid | 2-Acetoxybenzoic acid | Acetylsalicylate | Acetylsalicylsäure | acide 2-(acétyloxy)benzoïque | Acide acétylsalicylique | ácido acetilsalicílico | Acidum acetylsalicylicum | ASA | Aspirin | Aspirina | Azetylsalizylsäure | o-acetoxybenzoic acid | O-acetylsalicylic acid | o-carboxyphenyl acetate | Polopiryna | Salicylic acid acetate', 'atc': 'B01AC06', 'SideEffect': 'Respiratory alkaloses'}
{'Drug Name': 'Acetylsalicylic acid', 'synonyms': '2-Acetoxybenzenecarboxylic acid | 2-Acetoxybenzoic acid | Acetylsalicylate | Acetylsalicylsäure | acide 2-(acétyloxy)benzoïque | Acide acétylsalicylique | ácido acetilsalicílico | Acidum acetylsalicylicum | ASA | Aspirin | Aspirina | Azetylsalizylsäure | o-acetoxybenzoic acid | O-acetylsalicylic acid | o-carboxyphenyl acetate | Polopiryna | Salicylic acid acetate', 'atc': 'B01AC06', 'SideEffect': 'Systemic anaphylactic reaction'}
{'Drug Name': 'Acetylsalicylic acid', '